In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def logic_win(x,x_r):
    return (x >= x_r[0]) & (x < x_r[1])

#### Import and clean dataset, select out London

In [2]:
path = 'combined_collisions_v2.csv'

df = pd.read_csv(path,low_memory=False)
df.drop(columns='Unnamed: 0',inplace=True)

In [3]:
#Time column has some nans in it, must drop those entries

def is_str(x):
    Nx = len(x)
    I = np.ones(Nx).astype(bool)
    for ii in range(Nx):
        if type(x[ii]) != str:
            I[ii] = False
    return I

df = df.loc[is_str(df.Time.values),:]
df.aadf_Count_point_id = df.aadf_Count_point_id.values.astype(int)

In [4]:
#make hour, year and month columns
    #hour will be rounded 
def get_hour(T):
    Nt = len(T)
    H = np.zeros(Nt).astype(int)
    for ii in range(Nt):
        s = T[ii].split(':')
        h = int(s[0])
        m = int(s[1])
        if m > 30:
            h += 1
        H[ii] = h % 24
    return H

def get_ymd(D):
    Nd = len(D)
    # |
    ymd = np.zeros((Nd,3)).astype(int)
    for ii in range(Nd):
        s = D[ii].split('-')
        for nn in range(3):
            ymd[ii,nn] = int(s[nn])
    return ymd

def reformat_aadf_year(Year):
    Ny = len(Year)
    y = np.zeros(Ny).astype(int)
    for ii in range(Ny):
        y[ii] = int(Year[ii].split('-')[0])
    return y

df[['y','m','d']] = get_ymd(df.Date.values)
df['h'] = get_hour(df.Time.values)
#df['aadf_Year'] = reformat_aadf_year(df.aadf_Year.values)

##### Below selects out region. You can use Dan's feature to select Greater London, or code below for central London.
- additionally I drop all entries with no traffic counter matches

In [7]:
#central London box
Lat_r = [51.475,51.543]
Lon_r = [-0.179,0.023]

I_lat = (df.Latitude > Lat_r[0]) & (df.Latitude < Lat_r[1])
I_lon = (df.Longitude > Lon_r[0]) & (df.Longitude < Lon_r[1])
I = I_lat & I_lon

#df_lond = df.loc[I,:].copy() #selects central London
df_lond = df.loc[df.in_london,:].copy() #this selects Greater London

In [8]:
#drop all data that don't have traffic matches

I = df_lond.match == True
df_lond = df_lond.loc[I,:]
df_lond.aadf_Year = reformat_aadf_year(df_lond.aadf_Year.values)

#### Make neighborhoods feature

In [9]:
#Make a 3 (Lat, N/S) x 6 (Lon, E/W) grid
Nlat = 3
Nlon = 6
Nneigh = Nlon*Nlat

edges_lat = np.linspace(*Lat_r,Nlat+1)
dlat = np.diff(edges_lat)[0]
bins_lat = edges_lat[:-1]+0.5*dlat
edges_lon = np.linspace(*Lon_r,Nlon+1)
dlon = np.diff(edges_lon)[0]
bins_lon = edges_lon[:-1]+0.5*dlon

coord_neigh = [None]*Nneigh

df_lond['neigh'] = 0 #initialize neighborhood feature
ir = 0 #these are lat bins (rows)
ic = 0 #lon bins
for ii in range(Nneigh):
    Ilat = logic_win(df_lond.Latitude,(edges_lat[ir],edges_lat[ir+1]))
    Ilon = logic_win(df_lond.Longitude,(edges_lon[ic],edges_lon[ic+1]))
    I = Ilat & Ilon
    df_lond.loc[I,'neigh'] = ii
    
    coord_neigh[ii] = np.array([bins_lat[ir],bins_lon[ic]])
    
    ic+=1
    if ic == Nlon:
        ic = 0
        ir+=1

In [10]:
#Make a feature for if there is a match to a CLT feature or not (is there a bike lane or not?)

Inan = np.isnan(df_lond.aadf_CLT_SHARED.values)
df_lond['bikelane'] = np.ones(df_lond.shape[0]).astype(bool)
df_lond.loc[Inan,'bikelane'] = False

#replace NaNs in aadf_CLT data with -9 (NaNs don't work well with some statistical functions)
    #also reformat data types to int
num_nan = -9
f_cast2int = ['aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME']

for f in f_cast2int:
    df_lond.loc[Inan,f] = num_nan
    df_lond[f] = df_lond[f].values.astype(int)

In [13]:
traf_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().index)
Ninc_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().values)

######################################
Nthresh = 1 #this sets which traffic counters are included, they must have at least this many accidents 
######################################

I = Ninc_IDs >= Nthresh #this will select roads to include in the new DataFrame based on if they had at least this many accidents
I.sum()

IDs_df = traf_IDs[I]
Nid = len(IDs_df)

df_road = pd.DataFrame({'ID':IDs_df})
slight_rd = np.zeros(Nid).astype(int)
ser_rd = np.zeros(Nid).astype(int)
total_rd = np.zeros(Nid).astype(int)
for nn,ii in enumerate(IDs_df):
    I = df_lond.aadf_Count_point_id.values == ii
    total_rd[nn] = I.sum()
    slight_rd[nn] = df_lond.loc[I,'slight'].values.sum()
    ser_rd[nn] = total_rd[nn] - slight_rd[nn]
df_road['total'] = total_rd
df_road['slight'] = slight_rd
df_road['serious'] = ser_rd

safety_mode = ['neigh','Road_name','Road_name2','h','Day_of_Week','m','Speed_limit','Junction_Detail','Junction_Control','Pedestrian_Crossing-Physical_Facilities']
aadf_av = ['distance_to_cp','aadf_Pedal_cycles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
aadf_mode = ['bikelane','aadf_Road_name','aadf_Road_category','aadf_Road_type','aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME','aadf_Link_length_miles']

for f in safety_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]

for f in aadf_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]
        
for f in aadf_av:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mean()
        
df_road['total_norm'] = df_road['total'].values/df_road['aadf_Pedal_cycles'].values
df_road['serious_ratio'] = df_road['serious'].values/df_road['total'].values

C:\Users\mccracken.96\AppData\Local\Temp\1\ipykernel_18764\619031456.py:53: RuntimeWarning: divide by zero encountered in divide
  df_road['total_norm'] = df_road['total'].values/df_road['aadf_Pedal_cycles'].values


In [14]:
#add in aadf year availability metadata 
path_year = 'aadf_year_meta.csv'
df_year = pd.read_csv(path_year)

#merge the metadata into df_road
feat_add = list(df_year.columns)
feat_add.remove('Count_point_id')
for c in feat_add:
    df_road[c] = np.zeros(Nid).astype(int)
    for ii,ID in enumerate(df_road.ID.values):
        I = df_year.Count_point_id == ID
        df_road.loc[ii,c] = df_year.loc[I,c].values

#### Here is the final DataFrame, feel free to save your own version

In [15]:
df_road.sample(20)

,ID,total,slight,serious,neigh,Road_name,Road_name2,h,Day_of_Week,m,...,aadf_All_motor_vehicles,aadf_All_HGVs,aadf_LGVs,aadf_Buses_and_coaches,total_norm,serious_ratio,N_year,Year_start,Year_end,Max_year_gap
1285,47596,6,6,0,0,A2015,A2015,0,6,5,...,8399.500000,110.500000,549.500000,724.333333,0.065336,0.000000,22,2000,2021,1
474,8542,24,21,3,0,A118,U,18,2,3,...,39246.708333,663.208333,3551.583333,1216.333333,0.058300,0.125000,22,2000,2021,1
1364,38569,5,5,0,0,A232,C133,9,1,9,...,15376.600000,221.000000,1773.000000,83.000000,0.053879,0.000000,22,2000,2021,1
337,47730,31,24,7,6,A4209,C,19,5,5,...,22096.741935,391.741935,2407.483871,1078.354839,0.037073,0.225806,22,2000,2021,1
1412,6898,5,5,0,0,A205,A205,0,2,6,...,32897.000000,1533.400000,3187.200000,543.600000,0.006158,0.000000,22,2000,2021,1
991,46818,9,8,1,0,A222,C,18,3,5,...,11092.333333,257.111111,1449.777778,612.444444,0.024071,0.111111,22,2000,2021,1
1315,46464,6,6,0,0,A41,A41,19,2,3,...,24973.666667,671.666667,2994.500000,96.166667,0.063380,0.000000,22,2000,2021,1
609,8090,19,17,2,0,A437,U,8,3,4,...,20182.105263,450.631579,2243.105263,271.526316,0.176874,0.105263,22,2000,2021,1
1245,58156,6,5,1,0,A5,U,8,3,1,...,21977.333333,587.333333,2287.000000,923.000000,0.035964,0.166667,22,2000,2021,1
1200,17175,7,7,0,0,A504,U,8,3,2,...,15251.857143,122.000000,1167.000000,265.285714,0.063389,0.000000,22,2000,2021,1


In [ ]:
##save dataframe

# path = 'df_road_v2.csv'
# df_road.to_csv(path,index_label=False)